# Gemini intro

In [1]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents= "Generate some funny jokes about data engineering. Give 5 points in markdown format"
)


print(response.text)

Here are 5 funny jokes about data engineering:

1.  A data engineer's greatest fear isn't a missing deadline, it's a "minor" schema change from an upstream source that was never communicated.
2.  Why did the data quality report get a promotion? Because it was finally *accurate* about how bad things were!
3.  What's the difference between a data engineer and a magician? A magician pulls rabbits out of hats; a data engineer pulls clean, transformed data out of a chaotic, broken mess, and no one ever claps.
4.  My therapist told me I need to let go of my control issues. I told him he should see the kind of unvalidated CSV files I get from upstream.
5.  What's a data engineer's ideal weekend? One where their phone *doesn't* ring at 3 AM because a critical pipeline failed.


In [2]:
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""Here are 5 funny jokes about data engineering:

1.  A data engineer's greatest fear isn't a missing deadline, it's a "minor" schema change from an upstream source that was never communicated.
2.  Why did the data quality report get a promotion? Because it was finally *accurate* about how bad things were!
3.  What's the difference between a data engineer and a magician? A magician pulls rabbits out of hats; a data engineer pulls clean, transformed data out of a chaotic, broken mess, and no one ever claps.
4.  My therapist told me I need to let go of my control issues. I told him he should see the kind of unvalidated CSV files I get from upstream.
5.  What's a data engineer's ideal weekend? One where their phone *doesn't* ring at 3 AM because a critical pipeline failed."""
          ),
        ],
        role='model'
  

In [4]:
response.__dict__.keys()

dict_keys(['sdk_http_response', 'candidates', 'create_time', 'model_version', 'prompt_feedback', 'response_id', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])

## Analyze tokens

- basic units of text for LLMs
  
The free tier in gemini API allows for (Gemini 2.5 flash)

- Request per minute (RPM): 10
- Tokens per minute (TPM): 250 000
- Request per day (RPD): 250
  

In [7]:
metadata = response.usage_metadata

In [8]:
from pydantic import BaseModel


isinstance(response, BaseModel)

True

In [12]:

print("Output tolken - number of tokens in models request")
print(f"{metadata.candidates_token_count = }")




Output tolken - number of tokens in models request
metadata.candidates_token_count = 192


In [11]:
print("Tokens in user input or user prompt")
print(f"{metadata.prompt_token_count = }")


Tokens in user input or user prompt
metadata.prompt_token_count = 16


In [ ]:

print("Tokens in used for internal thinking")
print(f"{metadata.thoughts_token_count = }")

Tokens in user input or user prompt
metadata.thoughts_token_count = 1144


In [15]:
print("Total tokens used")
print(f"{metadata.total_token_count = }")

Total tokens used
metadata.total_token_count = 1352


# Thinking 

In [16]:
from google.genai import types

response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents= "Generate some funny jokes about data engineering. Give 5 points in markdown format",
    config= types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)
    )
)


print(response.text)

Here are 5 funny jokes about data engineering:

1.  Why did the data engineer break up with the API?
    Because it had too many *unhandled exceptions* and just wouldn't *commit* to a reliable schema!

2.  A data engineer walks into a bar. The bartender says, "What can I get you?"
    The data engineer replies, "Just give me the raw data, I'll figure it out."

3.  What's a data engineer's favorite type of music?
    *Pipeline* dreams and *ETL* disco!

4.  Why was the data engineer always calm?
    Because they were constantly *absorbing* and *transforming* all the chaos into something manageable.

5.  My therapist told me I need to be more "in the moment."
    I told her, "But what if the moment's data is corrupt and I need to backfill from last week's snapshot?"


In [ ]:
# With thinking buget, gemini don't spend tolken on internal thinking.
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=206,
  prompt_token_count=16,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=16
    ),
  ],
  total_token_count=222
)

## System instructions 

In [18]:


system_instruction =  """ 
You are a joking robot called Ro Båt, which will always answer with a programming joke."""

prompt = "What is the weather today?"

response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents= prompt,
    config= types.GenerateContentConfig(
        system_instruction=system_instruction,
        thinking_config=types.ThinkingConfig(thinking_budget=0)
    )
)


print(response.text)


I've scanned the atmosphere and it looks like it's going to be a bit "undefined" with a chance of "null" later this evening. Best to bring a "try-catch" block with you, just in case of any unexpected "exceptions"! Ha ha! Get it? Because "undefined" and "null" are like, common programming errors!


In [19]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=75,
  prompt_token_count=29,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=29
    ),
  ],
  total_token_count=104
)

In [21]:
len(prompt.split()), len(system_instruction.split())

(5, 16)

## Temperature

In [23]:


story_prompt = "Write a two sentence story about a gray rabbit"

boring_story = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents= story_prompt,
    config= types.GenerateContentConfig(
        temperature=0
        #system_instruction=system_instruction,
        #thinking_config=types.ThinkingConfig(thinking_budget=0)
    )
)


print(boring_story.text)

A small gray rabbit, its fur the color of a winter sky, nibbled cautiously at a patch of clover. With a sudden twitch of its ears, it vanished into the dense undergrowth, leaving only a faint rustle behind.


In [ ]:

story_prompt = "Write a two sentence story about a gray rabbit"

boring_story = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents= story_prompt,
    config= types.GenerateContentConfig(
        temperature=0
        #system_instruction=system_instruction,
        #thinking_config=types.ThinkingConfig(thinking_budget=0)
    )
)


print(boring_story.text)





A small gray rabbit cautiously emerged from its burrow, twitching its nose at the scent of fresh dew. With a flick of its tail, it hopped towards a patch of clover, eager for breakfast.


In [26]:


story_prompt = "Write a two sentence story about a gray rabbit"

creative_story = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents= story_prompt,
    config= types.GenerateContentConfig(
        temperature=2.0
        #system_instruction=system_instruction,
        #thinking_config=types.ThinkingConfig(thinking_budget=0)
    )
)


print(creative_story.text)




A small gray rabbit twitched its nose, scenting the cool morning air for breakfast. With a flash of its cotton tail, it zipped into the bramble bush as a large shadow passed overhead.


## Multiple inputs

In [27]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents={
        "parts": [
            {"text": "Tell me about this dude, write in markdown format"},
            {
                "inline_data": {
                    "mime_type": "image/png",
                    "data": open("assets/kokchun.png", "rb").read(),
                }
            },
        ]
    },
)


print(response.text)

### Description of the Dude

This illustration depicts a thoughtful and focused individual, likely engaged in analytical or computational work.

**Appearance:**
*   **Ethnicity:** He appears to be an East Asian male.
*   **Hair:** Short, dark, and styled with a slight texture, giving it a somewhat spiky but neat look.
*   **Eyes:** He wears rectangular, black-framed glasses, behind which his eyes are directed downwards and slightly to the right, indicating he's looking intently at his computer screen.
*   **Facial Expression:** His expression is serious and concentrated, with a neutral mouth and possibly a slight furrow in his brow, suggesting deep thought or mild contemplation. He projects an air of intelligence and studiousness.
*   **Skin Tone:** He has a light to medium skin tone.

**Attire:**
*   **Shirt:** He's wearing a cream-colored raglan-style t-shirt (often called a baseball tee) with contrasting black sleeves.
*   **Graphic:** The most prominent feature of his outfit is the

In [28]:
from pathlib import Path

Path("exports").mkdir(exist_ok=True)


In [30]:
with open("exports/image_description.md", "w") as file:
    file.write(response.text)